# BƯỚC 8 & 9: NÂNG CẤP SIÊU CẤP - SHUFFLENETV2 WITH CBAM ATTENTION
--- 
### 1. Mục tiêu tối thượng
- **Yêu cầu:** Nhận diện chính xác các vùng bệnh nhỏ nhất, vượt qua hiệu suất của mô hình ConvNeXtTiny.
- **Công nghệ:** Tích hợp **CBAM (Convolutional Block Attention Module)** vào ShuffleNetV2 để mô hình có khả năng 'tập trung ánh nhìn' vào vết bệnh.

### 2. So sánh đặc tính kỹ thuật

| Thành phần | ShuffleNetV2 (Cải tiến cũ) | **ShuffleNetV2 + CBAM (MỚI)** |
| :--- | :--- | :--- |
| **Attention Module** | Không có | **CBAM (Channel + Spatial)** - Tập trung vùng bệnh nhỏ |
| **Khả năng quan sát** | Toàn cục (Global) | **Địa phương (Local-Focus)** - Soi chi tiết các đốm bệnh |
| **Độ ổn định** | Trung bình | **Rất cao** (Bỏ qua nhiễu phông nền, Logo) |
| **Optimizer** | AdamW | **AdamW + OneCycleLR** (Tối ưu nhất hiện nay) |

In [ ]:
import os, torch, json, cv2, random, numpy as np, matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, confusion_matrix
import seaborn as sns
from tqdm import tqdm

# --- ĐỊNH NGHĨA MODULE ATTENTION CBAM ---
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        concat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(concat))

# --- GHÉP CBAM VÀO SHUFFLENETV2 ---
class CBAMShuffleNetV2(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        base = models.shufflenet_v2_x1_0(weights='DEFAULT')
        self.conv1 = base.conv1
        self.maxpool = base.maxpool
        self.stage2 = base.stage2
        self.stage3 = base.stage3
        self.stage4 = base.stage4
        self.conv5 = base.conv5
        self.ca = ChannelAttention(464) 
        self.sa = SpatialAttention()
        self.fc = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        x = self.conv1(x); x = self.maxpool(x)
        x = self.stage2(x); x = self.stage3(x); x = self.stage4(x)
        x = x * self.ca(x); x = x * self.sa(x)
        x = self.conv5(x); x = x.mean([2, 3]); x = self.fc(x)
        return x

## BƯỚC 8: Huấn luyện với Kỹ thuật Tinh chỉnh và Attention

In [ ]:
BASE_PATH = r'd:\HUTECH\AI\DeepLearning\DAHS\MangoLeaf'
DST_PATH = os.path.join(BASE_PATH, 'dataset_scientific_split')
RESULT_PATH = os.path.join(BASE_PATH, 'Result', 'ShuffleNetV2_CBAM_Improved')
os.makedirs(RESULT_PATH, exist_ok=True)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32; EPOCHS = 30

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_loader = DataLoader(datasets.ImageFolder(os.path.join(DST_PATH, 'train'), train_transform), batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(os.path.join(DST_PATH, 'val'), val_transform), batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(datasets.ImageFolder(os.path.join(DST_PATH, 'test'), val_transform), batch_size=BATCH_SIZE, shuffle=False)

CLASS_NAMES = datasets.ImageFolder(os.path.join(DST_PATH, 'train')).classes
NUM_CLASSES = len(CLASS_NAMES)

model = CBAMShuffleNetV2(NUM_CLASSES).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=0.00005, weight_decay=0.01)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.0006, steps_per_epoch=len(train_loader), epochs=EPOCHS)
scaler = torch.cuda.amp.GradScaler()

best_acc = 0
for epoch in range(EPOCHS):
    model.train(); tr_loss = 0
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(inputs); loss = criterion(outputs, labels)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        scheduler.step(); tr_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    model.eval(); v_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            out = model(inputs.to(DEVICE)); v_correct += (out.argmax(1) == labels.to(DEVICE)).sum().item()
    
    val_acc = v_correct / len(val_loader.dataset)
    print(f'>>> Val Acc: {val_acc:.4f}')
    if val_acc >= best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), os.path.join(RESULT_PATH, 'best_shufflenet_cbam.pth'))
        print('Saved Best Attention Model!')

## BƯỚC 9: Đánh giá chi tiết hiệu năng cải tiến

In [3]:
model.load_state_dict(torch.load(os.path.join(RESULT_PATH, 'best_shufflenet_cbam.pth')))
model.eval(); y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.to(DEVICE)); preds = outputs.argmax(1)
        y_true.extend(labels.numpy()); y_pred.extend(preds.cpu().numpy())

print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8)); sns.heatmap(cm, annot=True, fmt='d', xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES); plt.show()

# Lưu báo cáo cuối cùng
report_data = classification_report(y_true, y_pred, target_names=CLASS_NAMES, output_dict=True)
with open(os.path.join(RESULT_PATH, 'report.json'), 'w') as f: json.dump(report_data, f, indent=4)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9080\1576763042.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(RESULT_PATH, 'be

                  precision    recall  f1-score   support

     Anthracnose       0.97      1.00      0.99        39
Bacterial_Canker       1.00      1.00      1.00        40
  Bacterial_Spot       0.98      1.00      0.99        40
  Cutting_Weevil       1.00      1.00      1.00        40
        Die_Back       1.00      1.00      1.00        41
      Gall_Midge       1.00      0.95      0.97        40
         Healthy       1.00      1.00      1.00        40
  Powdery_Mildew       1.00      1.00      1.00        40
     Sooty_Mould       1.00      1.00      1.00        40

        accuracy                           0.99       360
       macro avg       0.99      0.99      0.99       360
    weighted avg       0.99      0.99      0.99       360



NameError: name 'confusion_matrix' is not defined